In [1]:
import os, re
import win32com.client

import numpy as np
from konlpy.tag import Kkma
from PIL import Image
from collections import Counter
from wordcloud import ImageColorGenerator, WordCloud

import matplotlib.pyplot as plt
%matplotlib inline


# file path를 정의합니다.
dir_path = os.getcwd()
file_path = dir_path + "\src\conversation.docx"

# 꼬꼬마 형태소 분석기 생성
pos = Kkma()

# For Korean characters
FONT_PATH = 'C:/Windows/Fonts/나눔스퀘어라운드/NanumSquareRoundL.ttf' 

In [52]:
def context_exporter(filepath):
    
    # win32com module object 생성 및 docx 파일 열기
    word = win32com.client.Dispatch("Word.Application")
    wb = word.Documents.Open(file_path)
    
    # docx 파일 선택
    doc = word.ActiveDocument
    print("선택된 파일 : ", doc)
    
    # docx 파일 내 text context를 가져옵니다.
    context = doc.Range().Text
    
    return context


def nouns_exporter(context_list):

    # nouns 리스트 생성
    nouns_list = []
    
    # 파싱을 완료한 데이터에서 한 context씩 꺼내서 konlpy 형태소 분석기로 형태소 분석 후 nouns 리스트에 저장
    for idx, item in enumerate(context_list):
        print("--------------------------------")
        print("순서 : ", idx)
        print("context : ", item)

        pos_list = []
        pos_list = pos.nouns(item)

        if not pos_list:
            # print("Empty item")
            pass
        else:
            for i in range(len(pos_list)):
                nouns_list.append(pos_list[i])
        
        print("현재까지 모아진 nouns 수 :", len(nouns_list))

    return nouns_list

def stopwording(nouns):
    print("=== stopwording ===")
    
    
    # stopwording(불용어 사전으로 필터링)
    with open('./src/stopwords.txt', 'r', encoding='utf8') as f:
        stopwords = f.read()
    
    print("stopwords :", stopwords)
    print("Origin nouns length is ",len(nouns))
    print("==================")
    
    # unique nouns 추출
    unique_nouns = set(nouns)
    
    # stopwords에 unique nouns가 존재하면 제거
    for word in unique_nouns:
        if word in stopwords:
            while word in nouns: nouns.remove(word)
    
    print("\nChanged nouns length is ",len(nouns))
    return nouns

In [3]:
# 1. 텍스트 파일 추출
context = context_exporter(filepath = file_path)
print("\n=== 추출완료 === \n\n", context)

선택된 파일 :  conversation.docx

=== 추출완료 === 




In [5]:
# 2. 특수문자 및 기타 오타등의 불필요한 요소 파싱
parse = re.sub('[-=.#/:$}()-.‘’!?-“\x0c”ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅍㅠㅎ<#~;…\n]','',context)

# 띄워쓰기 제거
parsed_context = parse.replace('\n','')
print("=== 파싱완료 === \n\n", parsed_context)

# parsing된 context 내에서 '\r'를 기준으로 split 합니다.
context_list = parsed_context.split('\r')

# context_list 내에서 빈 item를 제거합니다.
context_list = list(filter(None, context_list))

=== 파싱완료 === 




In [16]:
# 3. 형태소 분석
nouns = nouns_exporter(context_list)

--------------------------------
순서 :  0
context :  발제
현재까지 모아진 nouns 수 : 1
--------------------------------
순서 :  1
context :  영화의 첫인상이 어땟는지 한 두줄로 적어주세요
현재까지 모아진 nouns 수 : 5
--------------------------------
순서 :  2
context :  왜인생의 한 파트를 '버스타기'라고 표현했을가요
현재까지 모아진 nouns 수 : 12
--------------------------------
순서 :  3
context :  왜       라고 표현했는지 생각해보세요
현재까지 모아진 nouns 수 : 15
--------------------------------
순서 :  4
context :  주인공 크리스의 말에          '     라는 장면이 나옵니다 영화 초입 감독이 어떤 느낌을 나타내고자 했을가요
현재까지 모아진 nouns 수 : 24
--------------------------------
순서 :  5
context :  어떤 아저씨가 골밀도 측정기를 타임머신이라고 부릅니다 무엇을 표현하고자 했을까요
현재까지 모아진 nouns 수 : 32
--------------------------------
순서 :  6
context :  아들이랑 농구하던 크리스는 아들에게 꿈을 가지고 그걸 보호하라고 합니다 심지어 아빠 자신까지도 그걸 못한다고 해도 말입니다 우리도 살면서 이상과 현실의 대립사이에서 꿈을 지킬 수 있나요
현재까지 모아진 nouns 수 : 51
--------------------------------
순서 :  7
context :   
현재까지 모아진 nouns 수 : 51
--------------------------------
순서 :  8
context :  영화에선                 동의하는지 않하는지 자신의 생각과 함께 말해주세요
현재까지 모아진 no

현재까지 모아진 nouns 수 : 508
--------------------------------
순서 :  73
context :  다희 슬라임 영상  블로그 쓰면서 지난 여행을 돌아보는 것  여행 당시도 좋지만 다시 생각해 보는 것도 좋은 것 같다 
현재까지 모아진 nouns 수 : 515
--------------------------------
순서 :  74
context :  알바 끝나고 집에 가면서 안락한 전기장판 귤 핸드폰 그런 것들을 생각하는 것 
현재까지 모아진 nouns 수 : 515
--------------------------------
순서 :  75
context :  예은 핵심기억에 대해 생각해보면서 나는 미래를 향해 살고 있다고 느꼈다 좌우명이 후회하지 말자여서인지 핵심기억이라기보다는 과거에 좋았다 정도로 생각하는 것 같다 사람마다 다른 시간에서 살아간다는 말이 와 닿았다 힘든 일이 있을 때도 이것 또한 지나가겠지 라고 생각하며 지나가는데 여기서도 미래지향적인 사람인 것 같다고 생각이 들었다 
현재까지 모아진 nouns 수 : 543
--------------------------------
순서 :  76
context :   핵심기억은 오히려 나에게 별로 중요하지 않은 사람들이 했던 말이 오래 남는 것 같은 느낌이 있다 
현재까지 모아진 nouns 수 : 551
--------------------------------
순서 :  77
context :  정습  2019년이 되고 이상적인 삶과 현실적인 삶에 대해 고민했는데 24살이 되고 좀 더 철들어야되나에 대해 생각을 했다 방학 동안 여유롭게 보내는게 아니라 학원에 다니고 공부를 열심히해야하나 생각도 들고 이런 생각이 들 때 주변 친구들은 시험 준비 자기 길을 찾아가는 것 같아 친구들은 이미 멀리 앞서 나간 것 같고 먼 미래에 나랑 사회적인 계급도 다르지 않을까라는 생각도 들었다 하지만 친구들이 잘 하고 있다고 위로해줘서 그 기억이 따뜻하게 남아있고 결국 조급하지 말고 

현재까지 모아진 nouns 수 : 1288
--------------------------------
순서 :  124
context :  장면과 적절 국어시간에 배우던 것과 달라 양적 증가가 끝에 닿으면 책 양적 감소가 끝에 닿으면 시 단어와 의미의 손실이 책보다 덜한 시 시인의 사유가 거대할수록 언어는 농축되고 무거워져 길이는 짧아지고 깊이는 가늠할 수 없다
현재까지 모아진 nouns 수 : 1314
--------------------------------
순서 :  125
context :  2 윤동주 시인과 그의 친구이자 독립운동가인 송몽규는 서로 크고 다양한 영향을 주고 받습니다 동갑내기에 같은 집에 살고 문인이라는 공통점을 가지지만 성격과 가치관 그리고 늘 정해져있던 1등과 2등의 굴레가 그들의 차이였죠 당신은 어떤 사람들과 어떤 관계를 맺으며 어떤 영향을 만들어가고 있나요
현재까지 모아진 nouns 수 : 1342
--------------------------------
순서 :  126
context :  다희  주변인의 영향을 많이 받는 편이다 좋게 말하면 흡수를 잘 하는 편이고 나쁘게 말하면 잘 쓸려다닌다는 거라고 할 수 있다 그래서 나만의 기준을 만들려고 노력한다 좋은 영향을 말하자면 첫번째로는 부모님이다 부모님을 닮아서 악바리 근성이 있다 공부가 아니라 아파도 알바가기 지각은 해도 결석은 안 하기 두번째로는 친한 친구인 김주희다 여러 터닝포인트를 겪었는데 함께 유럽 여행을 하면서 가장 많이 영향을 받은 것 같다 사고나 말투를 많이 닮게된 것 같다 마지막으로는 예은이다 최근에 느꼈는데 아는 언니를 만나서 취미에 대한 이야기를 하는데 취미가 많아진 것과 그를 즐기는 것 그리고 감성을 닮게 된 것 같다
현재까지 모아진 nouns 수 : 1383
--------------------------------
순서 :  127
context :  예은  다희언니의 좋은 오지랖을 닮게 되었다 최근에 여행가서 택시 아저씨에 대한 에피소드를 겪으며 더 

현재까지 모아진 nouns 수 : 2390
--------------------------------
순서 :  155
context :  1내면과 외면
현재까지 모아진 nouns 수 : 2394
--------------------------------
순서 :  156
context :  예은 내면 내면이 본선이고 외면이 예선 외면 피부하얗고 어깨가 제일 중요함 이 오빠는 어깨만 충족햇습니다 180넘는다고 하더라고요 내면 음식점갓을때 종업원에게 대하는 태도나 정습예히 공감능력을 갖추려고 노력하는 사람 저한테 질문이 많은 사람이 좋아용 오빠랑 얘기하면 오디오가 안비고 장난치는걸 좋아함 건대데이트 하면서도 계속 장난 걸엇어용 이사람 통한다 이느낌 그 오빠는 저한테 그런걸 느꼇대영  
현재까지 모아진 nouns 수 : 2436
--------------------------------
순서 :  157
context :  재환 내면 일부러 안썻다 뭔가 잇는 그대로 얘기하고 싶어성 역삼각형 모양같다 내면이 나의 스타일 내면을 내 스타일대로 받아들이겟다 중요하게 생각하는건 내면입니다 왜냐면 겉으로는 사람을 판단할 수 없으니께 내면으로는 긍정적 예의 성실함 첫인상임 또 거짓말은 하면 안된다고 생각함 아예 트루로 다 말하는 게 좋은것같다 모든건 결국 돌아와 외면적인건 이런모습은 싫다는 자기 관리 안하는사람 내가 예전에 살이 쪄봐서 느꼇지만 너무 게을럿고 귀차니즘이 심햇다 그래서 느낌쓰
현재까지 모아진 nouns 수 : 2484
--------------------------------
순서 :  158
context :  지용 외면 처음에는 내면이라고 생각햇지만 진지하게 다시 생각해보니 20살이후로 내가 엄청 좋아하는 사람이 없엇던듯하다 외면이 먼저 내 스타일이어야 본선 진출입니다 외면은 조금 수수한 사람 임수정처럼 여리여리한 느낌 전에 좋아햇던 세 분도 느낌이 그랫습니다 내적인건 나와 다른 사람은 호감이 가지 않아 취향이 비슷한 사람이 좋다 유머코드도 유머코드인데 영

현재까지 모아진 nouns 수 : 3290
--------------------------------
순서 :  196
context :  다희  마라탕을 먹었다 퇴직금이 들어왔습니다 200 와우 희원이는 400 와우 근데 200중에 120이 학원비 유유 국가장려금 신청 121만원까지 받을 수 있는데 엄마 줄 듯 효녀네 학원 시작 재밌음 영어회화학원 다닐 맛 남 무리가 생겨서 재밌음 나이대도 다양해 백발할아버지도 있다 취직생각 노노 용기를 받았다 평일 월금 아침9시에가서 2시까지 수업듣고 스터디까지하면 5시까지도 함 이디야 알바도 하는 중 7개월치 결제 265만원 알바는 토일 410시까지 성곡 사거리 지점 전주 영화제를 다녀옴 다행이에요 침묵의 장벽을 봄 다행이에요는 목포에서 어떤 좀 진짜 좀 나도 좀 몰랐는데 유명한 사람들이 모여서 목포에 괜찮은 마을을 차린건데 정부에서 지원을 받아서 뭐 힐링 프로젝트 도시를 알리고 뭐 그럼 근데 보다가 울었음 많이 상영했음 좋겠음 예은이랑 전시회도 가고 플리마켓 가고 희현언니 만나고 아 전주 여행도해서 한복도 입고 놀았어요 끝 아니고 자격증 필기 붙어서 실기 보러가는데 전라도 광주에서 봅니다 진짜 끝
현재까지 모아진 nouns 수 : 3411
--------------------------------
순서 :  197
context :  34분 늦은 재환이  다희는 행복해 보였는데 나는 너무 불행했다 맥날에서 감기에 걸림 감기가 안끝나요 감기를 동아리 사람들에게 다 옮겨서 미안하다 학교 프로젝트가 너무 쌓였다 자퇴 생각까지두 이제 전공은 안살릴래요 아깝지만 너무 힘듦 지금도 힘든데 회사가면 더 힘들지 않을까 공기업 준비중이긴 함
현재까지 모아진 nouns 수 : 3445
--------------------------------
순서 :  198
context :  다희  벌써 공부 중이야 추진력 쩐다
현재까지 모아진 nouns 수 : 3449
--------------------------------
순서 :  199
c

...

현재까지 모아진 nouns 수 : 6234
--------------------------------
순서 :  446
context :  3번
현재까지 모아진 nouns 수 : 6237
--------------------------------
순서 :  447
context :  지용  취미는 영화 축구 독서가 새로 생김 드디어 처음으로 8권이 넘었음 한달에 한권 일년에 열두권이 목표인데 한달에 한권 하구 있고 수업시간에도 하나 읽고있어서 괜찮구 이번엔 채울수 있을 거 같아여  파이팅 보드게임 6개월 지났느데 이제 눈을 떠서 드라마보기도 취미긴 한데 요즘 잘 안보다가 지정생존자 보고 타인은 지옥이다 보는중 멜로가 체질은 노래가 좋아서 봄
현재까지 모아진 nouns 수 : 6288
--------------------------------
순서 :  448
context :  다희  저는 진짜 꾸준히 하는 취미는 팝송듣기 음악  첨엔 2년전에 애플 뮤직에 무료로 함 쓰고 그 뒤에 까먹고 계속 구독햇는데 너무 너무 잘들어서 영화 보고 영화관에서 혼자보기 가 취미 진짜 예능보기 나는 뭔가 휴식시간이 주어졌을 때 수동적으로 받아드리는게 좋아여 방구석은 잘보고 전지적참견이랑  긁거 4번하고 나혼자 미우새 집사부  다 좋아여  롱보드 멋잇구 배드민턴 하고싶고 클라이밍 멋잇음  와우 나두 해보고 싶
현재까지 모아진 nouns 수 : 6339
--------------------------------
순서 :  449
context :  예은  저는 근데 하고싶은게 하도 많아서 어디까지 취미 인지 모르겟음
현재까지 모아진 nouns 수 : 6345
--------------------------------
순서 :  450
context :  일기 사진찍기  책읽기 기록하기 별거별거 많이 기록 검색도 좋아함 사람만나기혼자하는운동나사빠지느거도 취민가  그리고 이상형도 모으기도 취민데지용이형 이상형을 못모아서있으면 매칭점 스겜 인터뷰보는걸 좋아함 작가님들 인터뷰도 찾아보고 

현재까지 모아진 nouns 수 : 7129
--------------------------------
순서 :  503
context :  현선  저는 상상을 너무 많이해서 다른사람들도 다 그렇게 하는줄 알았느데  그래서 막 그냥 그때 그때 순간 순간 상상을 너무너무 많이 해여  알아서 하래 상상을 진짜 막 
현재까지 모아진 nouns 수 : 7142
--------------------------------
순서 :  504
context :  소설도 보면 이것저것 하고 사진 보면 이것저것 아이디어 하고 
현재까지 모아진 nouns 수 : 7146
--------------------------------
순서 :  505
context :  영어공부할때는 외국인들이랑 상상하면서 영어대회하는 생각하면서 혼자 웅얼웅얼도 하고 
현재까지 모아진 nouns 수 : 7157
--------------------------------
순서 :  506
context :  영어버젼으로 가 아닐까 생각 여행 갈떄 그 당시에 연애인들 공유를 만나느거에여 공유가 도깨비가 여행을 가는거야 내친구들이랑 자기랑 같이 여행 걔네가 렌터카 빌려서 같이 다니고 어쩔땐 내 일기장을 가지고 봐서 노래 만드는 것도 상상 해보고 여행가보고싶을 땐 나영석이 갑자기 와서 꽃청춘을 찍자고 연락오고 별별상상을 너무 해여 
현재까지 모아진 nouns 수 : 7188
--------------------------------
순서 :  507
context :  언니 스타일은 월터  이터널션사인 빼고 다 봤음 방구셕 방구셕  키키킼 발음 구림 키키키킼 발음 구리데여  이상한 상상도 너무 하고 이상한거 같아여 이누나
현재까지 모아진 nouns 수 : 7214
--------------------------------
순서 :  508
context :  재환  암 레디  상상을 옛날에 되게 뚱뚱해서 그때 부터  살을 빼야겟다라는 생각을 햇어여 그냥 딱 운동을 하다가도 힘들어도 생각을 해 살을 뺀 나를 상

현재까지 모아진 nouns 수 : 8047
--------------------------------
순서 :  565
context :  예은  독립 집구하기 새로운시작
현재까지 모아진 nouns 수 : 8056
--------------------------------
순서 :  566
context :  정말 정신없는 하루하루를 보냈습니다 지난 영화모임에 빠진 이유가 이때로 밀렸기 때문이지요 그 이유는 바로 독립입니다 사정상 당장 한 달 이내로 집을 구해야 하는 상황이 되었고 그때부터 저에게 맞는 전세대출과 조건에 맞는 집을 알아보기 시작했어요
현재까지 모아진 nouns 수 : 8078
--------------------------------
순서 :  567
context :  집을 구한다는 게 이렇게 어려운 일인 줄 몰랐어요 나름 풍파를 겪으며 살아왔다고 느꼈는데 정말 찐풍파였습니다 집을 구하는 동안 23은 빠진 것 같아요 여러분을 만날 때면 다시 충전해서 갈 것입니다 지금도 빵 먹는 중
현재까지 모아진 nouns 수 : 8090
--------------------------------
순서 :  568
context :  일 하는 시간과 자는 시간을 제외하고 밥을 먹으면서도 각종 집 구하기 어플 보기  서울수도권부동산 전화돌리기  점심시간 혹은 퇴근 이후에 집 보러 다니기  새벽에는 전세대출과 부동산 용어 공부하기 끝에 결론적으로는 제가 받으려는 대출에 맞는 집을 구할 수 있었고 임대차 계약서 작성과 대출 신청까지 완료하여 지금은 대출 심사를 기다리고 있습니다
현재까지 모아진 nouns 수 : 8124
--------------------------------
순서 :  569
context :  말고도 꽤 여러 일들이 있었지만 최근에 생각나는 거라고는 오직 집뿐이네요 아직도 집 얘기만 하면 두통과 복통이 찾아온답니다 혹시 나중에 전세대출 하실 분이 있으시다면 제가 도와줄 수 있는 선에서는 도와드릴게요 아무도 없이 혼자 하나부터 열까지 다 하려니

현재까지 모아진 nouns 수 : 8838
--------------------------------
순서 :  600
context :  3 현선
현재까지 모아진 nouns 수 : 8840
--------------------------------
순서 :  601
context :  3 캣은 독립적인 삶을 위해 일부러 집에서 멀리 떨어진 대학에 진학했는데요 앞으로 캣의 삶이 어떻게 나아갈지 함께 상상해보아요
현재까지 모아진 nouns 수 : 8849
--------------------------------
순서 :  602
context :  3
현재까지 모아진 nouns 수 : 8850
--------------------------------
순서 :  603
context :  현선  지금쯤 캣은 어떤 삶을 살고 있을까요 밴드를 시작했을지 패트릭과는 아직도 만나고 있을지 어떤 공부를 하고 어떤 직업을 선택해 살고 있을지 아주 궁금한 마음이 들어요 제가 상상해봤을 때 캣은 적어도 대학 생활동안은 밴드 활동을 하며 깊은 내용이 담긴 노래들을 써냈을 것 같고 밴드 활동을 지속하진 않더라도 자신의 목소리를 내는 직업을 선택해 살고 있지 않을까 싶어요 패트릭과는 헤어졌을 것 같다는 생각이 들고 하지만 나중에 서로 30대 중후반쯤 자신들이 원하던 직업에서 어떤 자리를 잡았을 때 둘이 만나게 될 것 같은 상상을 
현재까지 모아진 nouns 수 : 8883
--------------------------------
순서 :  604
context :  예은  대학에 진학한 이후 몇 달 되지 않아 혹은 1년을 채 채우지 못하고 패트릭과는 헤어졌을 것 같아요 대학 생활은 등록금을 한 번도 내지 않고 과탑을 찍으며 열심히 잘 했을 것 같습니다 약간 후배들이 저 언니 너무 멋있지 않아 하는 그런 언니 하지만 막상 말 걸기에는 조금 무섭고 멀리서 동경만 하는 그런 언니
현재까지 모아진 nouns 수 : 8906
--------------------------------
순

현재까지 모아진 nouns 수 : 9494
--------------------------------
순서 :  640
context :  재환 좋아했던 걸 숨기게 된 경험은 전 여자친구 사귀었을 때 여자친구는 있지만 어디 사람이라고는 말하지 않았어 외국인은 처음 사귀기도 했고 일본과의 관계가 좋지는 않았으니까 말하기가 꺼려졌었어 근데 여자친구랑 헤어지고 나서 밝혔는데 친구들이 재환이 광복했다고 했었어 그리고 좋아했던 걸 싫어했던 경험은 제품을 너무 사랑하는데 광범위적으로 말하면 4차산업을 싫어하게 됐어 개인적으로 돈을 너무 많이 써 이번에 스피커 구글꺼를 새로 사고 크롬캐스트 그게 뭐야 쩐다  취업할 곳이 너무 힘들어졌다 알고리즘을 다루는 방법에 대해서 테스트를 보고 스카이프를 통해서 면접을 보고 옛날엔 간소했는데 요즘은 너무 복잡하다 그리고 너무 휴대폰에 지배당하는 느낌 그리고 싫어했던 것을 좋아한 경험은 요즘은 아날로그가 너무 땡긴다 그리고 아날로그가 좋은 제일 큰 이유는 핀테크가 너무 개발되어서 결제가 너무 쉬워져서 소비 과촉진을 불러일으키는 것 같아 그리고 제일 싫어했던 것은 운동인데 요즘은 너무 하는게 재미쏘 
현재까지 모아진 nouns 수 : 9556
--------------------------------
순서 :  641
context :  현선 저는 저는 저는 저는 저는저는저는 저는 좋아했던 것을 숨기게 된 경험은 정습이가 말해서 그런 건지도 모르겠지만 호감있는 사람이 있으면 인정하기 싫은 느낌 피곤해지고 그러니까 싫어했던 것을 좋아하게 된 경험은 고등학생 때는 가치있는 책의 기준이 고전이나 딱 있어보이는 책을 읽는 것이라고 생각했어 대중을 음해하는 행위 자체가 조금  대중문학 자체가 대중이 창작한 문학이고 그 안에는 그들의 인생이 있기 때문에  요즘은 내가 좋아하는 것을 많이 하려고 노력합니다아
현재까지 모아진 nouns 수 : 9588
--------------------------------
순서 :  642
context :  다희 

In [18]:
# 5. word counter test
c = Counter(nouns)

# 상위 100개 출력 테스트
print(c.most_common(100)) 


In [41]:
# 6. 불용어 제거 (stopwording)
nouns = stopwording(nouns)

=== stopwording ===
stopwords : 것 때 저 수 예 근데 현 거 지 중 데 내 뭐 전 애 그 라 바 리 편 듯 한 들 후 안 얘 걔 뭔가 둘 그걸 이 곳 줄 두 어캐 음 은 습 예 여 번 
Origin nouns length is  8911

Changed nouns length is  8911


In [42]:
# stopwording result
print(' '.join(nouns))

발제 영화 첫인상 땟 왜인 파트 버스 버스타기 타기 표현 가요 표현 생각 주인공 크리스 말 장면 영화 초입 감독 느낌 가요 아저씨 골 골밀도 밀도 측정기 타임머신 무엇 표현 아들 농구 크리스 꿈 그것 보호 보호하라 하라 아빠 자신 도 말 우리 이상 현실 대립 대립사이 사이 영화 동의 자신 생각 만약 당신 인터뷰 셔츠 바지 영화 주인공 주인공크리스 크리스 아들 아들크리토퍼 크리 토퍼 행동 말 만약 자신 아빠 엄마 공감 부분 추구 자신 삶 파트 버스 버스타기 타기 행복 변화 때문 생각 영화 영화속 속 삶 부분 버스 버스타기 타기 짓 행복 등 묘사 자신 모습 영화 외의 파트 자신 생각 행복 무엇 자신 직업적 꿈 장래 무엇 개인적 직업적 장래 장래희망 희망 선택 선택하라 하라 사회 직업 이상적 꿈 목표 목표등 등 근황 근황토크 토크 제주 제주도 제주도여행 도 여행 최고 게스트 게스트하우스 하우스 파티 아쉬움 맥 맥날 날 필터링 학교 학교특강 특강 임 새 채널 앞 영상 영상제작예쩡 제작 쩡 태국 일본 일본여행 여행 우 고취 고취준중 준중 영화 첫인상 땟 치열 소시민 생활 크리 크리스토퍼 스토퍼 자기 자기상황 상황 공감 선물 느낌 세상 다양 사람 시간대 일상 중함 인턴 바웃 바웃타임 타임 느낌 복합적 인생 파트 버스 버스타기 타기 표현 가요 종착역 추락 종착 역 인생 부분 끝 버스 감 인생 버스 기본적 권리 자기 선택 표현 행복 여정 표현 버스 일상 표현 반복 때문 아저씨 골 골밀도 밀도 측정기 타임머신 무엇 표현 다 측정기 옛날 자기 행복 추억 매개체 타임머신 생각 줌 노숙자 주인공 반대 황금 황금과거 과거 진취적 표현 생각 행복 사이 단어 생각 행복 을 추구 의미 미래 지금 행복하라 하라 너희 만 행복 나 의미 단어 철자 중요 행복 자체 발제 발제이외 이외 추가 추가적인 적인 내용 언어 언어유희 유희 의미 아내 크리 크리스토퍼 스토퍼 아내 크리 크리스토퍼 스토퍼 어린이 사랑 크리스 아빠 자기 충족 애기 밑 함 크리 크리스토퍼 스토퍼 아들 농구 크리스 꿈 그것 보호 보

In [43]:
noun_text = ''
for word in nouns:
    noun_text = noun_text +' '+word

In [46]:
# 원하는 아이콘으로 mask 제작
mask = np.array(Image.open("./src/movie_icon.png"))

In [47]:
print(mask)

[[26 26 26 ... 26 26 26]
 [26 26 26 ... 26 26 26]
 [26 26 26 ... 26 26 26]
 ...
 [26 26 26 ... 26 26 26]
 [26 26 26 ... 26 26 26]
 [26 26 26 ... 26 26 26]]


In [48]:
for i in range(len(mask)):
#     print(mask)
    for j in range(len(mask[i])):
        if mask[i][j] == 26:
            mask[i][j] = 255

In [54]:
print(mask)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [49]:
# image_colors = ImageColorGenerator(mask)

In [51]:
wordcloud = WordCloud(#max_font_size=400,
                      max_words = 300,
                      width = 800,height = 800,
                      background_color='white', 
                      mask = mask,
                      font_path=FONT_PATH).generate(noun_text)




NameError: name 'FONT_PATH' is not defined

In [55]:
# plotting
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

NameError: name 'wordcloud' is not defined

<Figure size 720x720 with 0 Axes>

In [75]:
# 생성한 wordcloud를 저장합니다.
wordcloud.to_file("wordcloud.png")

In [7]:
# problem = context_list[547]
# problem

# x = 0 
# for i in context_list:

          
#     if i == problem:       
#         print("문제있음")
#         print(x)
#         context_list.pop(x)
    
#     x += 1